# Ultrasound Simulations

In [ ]:
from typing import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from time import time
from tqdm import tqdm
from numba import jit
import math
from src.Scene import Scene

%matplotlib inline
plt.rcParams['figure.dpi'] = 140

In [ ]:
rx_pos = np.array([
    [ 0, 100],
    [10, 100],
    [20, 100]
])

tx_pos = np.array([
    [ 5, 100],
    [15, 100]
])

In [ ]:
def merge_positions(pos1, pos2):
    """
    Computes the cartesian product of two position spaces 
    """
    i = 0
    merged = np.empty([pos1.shape[0]* pos2.shape[0], pos1.shape[1]+pos2.shape[1]])
    for prx in pos1:
        for ptx in pos2:
            merged[i,:pos1.shape[1]] = prx
            merged[i,pos1.shape[1]:] = ptx
            i+=1
    return merged

pos = merge_positions(rx_pos, tx_pos)
pos.shape

In [ ]:
pos_rx_tx = pd.DataFrame(pos, columns=['r_x', 'r_z', 't_x', 't_z'])
pos_rx_tx

In [ ]:
def circle_positions(r_vec:np.ndarray, midpoint: np.ndarray, num: int) -> np.ndarray:
    m_cmplx = midpoint[0]+ 1.0j*midpoint[1]
    r_cmplx = r_vec[0] + 1.0j*r_vec[1]

    pos_elements = np.empty([num, 2])
    rotations = np.exp(2j*np.pi*np.arange(num)/num)
    pos_cmplx = rotations*r_cmplx + m_cmplx

    pos_elements[:, 0] = pos_cmplx.real
    pos_elements[:, 1] = pos_cmplx.imag

    return pos_elements


In [ ]:
circle_positions([10, 0], [10,60], 8)

In [ ]:


pos_scatter = pd.DataFrame(
    circle_positions([10, 0], [10,60], 100), 
    columns=['x','z'],
    dtype=np.float64)

pos_scatter

In [ ]:
proj_2d = np.array([[1, 0], [0, 0], [0,1]], dtype=float)

In [ ]:
ax = plt.subplot()

ax.scatter(pos_rx_tx.t_x, pos_rx_tx.t_z, label='TX')
ax.scatter(pos_rx_tx.r_x, pos_rx_tx.r_z, label='RX')
ax.scatter(pos_scatter.x, pos_scatter.z, label='scatter')
ax.set_ylim(0,120)
ax.set_xlim(0,120)

ax.legend()

In [ ]:
# Experiment Specs:

# Sampling freq.
f_samp = 100e6 # [Hz]

f_min, f_max = 2e6, 10e6 # [Hz]    <-- TODO

# Sampling period
dt_samp = 1/f_samp # [s]

# Mean speed of sound (water)
c_sound = 1_484_000.0 # [mm/s]

t_record = 2e-4 # [s]

In [ ]:
time_series= np.zeros([pos_rx_tx.shape[0], int(t_record*f_samp)])

In [ ]:
for i, row in enumerate(pos_rx_tx.to_numpy()):
    for scat in pos_scatter.to_numpy():
        r1 = np.linalg.norm(row[:2]-scat)
        r2 = np.linalg.norm(row[2:]-scat)

        idx =int((r1+r2)/c_sound*f_samp)
        # print(idx)
        time_series[i, idx] += 1/(r1*r2)

In [ ]:
ax = plt.subplot()
ax.plot(np.arange(time_series.shape[1])*dt_samp, time_series[0])
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_xlabel('time [s]')

In [ ]:
import numpy as np

class Simulation2D:

    def __init__(self, f_samp: float,  c: float, t1: float, t0: float=0.0):
        """Generating simulation data in a 2D coordinate system. (Still 3D physics where used for the simulation itself.)

        Args:
            f_samp (float): sampling frequency. unit=[s]
            c (float): speed of signal. unit=[mm/s]
            t1 (float): simulation stop time. unit=[s]
            t0 (float): simulation start time. unit=[s]
        """
        if t0 >= t1:
            raise Exception('t0 has to be smaller than t1')

        self.f_samp = f_samp
        self.t0 = t0
        self.t1 = t1
        self.c = c


    @property
    def sig_len(self) -> int:
        """Signal length (i.e. SIG_LEN). unit=none
        """
        return int(self.t_record * self.f_samp)

    @property
    def t_record(self) -> float:
        """duration of recording. unit=[s]
        """
        return self.t1 - self.t0


    def simulate(self, pos_tx_rx: np.ndarray, pos_scatter: np.ndarray) ->  np.ndarray:
        """[summary]

        Args:
            pos_tx_rx (np.ndarray): defining the positions of the TX and RX elements. shape=(NUM_REL, 4) & unit=[mm]
            pos_scatter (np.ndarray): defining the positions of the scatters. shape=(NUM_SCATTER, 2) & unit=[mm]

        Returns:
            np.ndarray: resulting timelines from simulation. shape=(NUM_REL, SIG_LEN) & unit=[s]
        """

        time_series = np.zeros([pos_tx_rx.shape[0], self.sig_len])

        for i, row in enumerate(pos_tx_rx):
            for scat in pos_scatter:
                r1 = np.linalg.norm(row[:2]-scat)
                r2 = np.linalg.norm(row[2:]-scat)

                idx =int((r1+r2)/self.c*self.f_samp)
                time_series[i, idx] += 1/(r1*r2)
        
        return time_series

    @property
    def time_space(self) -> np.ndarray:
        return np.arange(self.sig_len)/self.f_samp + self.t0
    

In [ ]:
s = Simulation2D(
    f_samp=100e6,
    c=1_484_000.0,
    t1=2e-4
)

In [ ]:
t = s.simulate(pos_rx_tx.to_numpy(), pos_scatter.to_numpy())

In [ ]:
ax = plt.subplot()
ax.plot(s.time_space, t[0])
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_xlabel('time [s]')